# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat.
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.

In [32]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [15]:
data = pd.read_csv('checkins.csv', sep = ';')

In [16]:
print data.columns

Index([u'id', u'user_id', u'venue_id', u'latitude', u'longitude',
       u'created_at'],
      dtype='object')


In [17]:
data.head()

id  user_id  venue_id   latitude   longitude            created_at
0  984222    15824      5222  38.895112  -77.036366   2012-04-21 17:43:47
1  984234    44652      5222  33.800745  -84.410520   2012-04-21 17:43:43
2  984291   105054      5222  45.523452 -122.676207   2012-04-21 17:39:22
3  984318  2146539      5222  40.764462 -111.904565   2012-04-21 17:35:46
4  984232    93870    380645  33.448377 -112.074037   2012-04-21 17:38:18

In [18]:
print data.shape
print data.dtypes

(396634, 6)
id              int64
user_id         int64
venue_id        int64
latitude      float64
longitude     float64
created_at     object
dtype: object


In [19]:
data = data[:100000]

In [20]:
#изменение типа столбца с помощью функции apply
data.created_at = data.created_at.apply(pd.to_datetime)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [21]:
X = data[['latitude', 'longitude']].values

In [24]:
print X.shape
print X[:5] 

(100000L, 2L)
[[  38.8951118  -77.0363658]
 [  33.800745   -84.41052  ]
 [  45.5234515 -122.6762071]
 [  40.764462  -111.904565 ]
 [  33.4483771 -112.0740373]]


In [25]:
ms = MeanShift(bandwidth=0.1)
ms.fit(X)

In [62]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique, counts_label_unique = np.unique(labels, return_counts=True)
n_clusters_ = len(labels_unique)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [65]:
# это индексы массива cluster_centers кластеров, в которых больше 15 эелементов
big_labels_unique = labels_unique[np.where(counts_label_unique > 15)]

In [66]:
big_cluster_centers = cluster_centers[big_labels_unique]

In [68]:
print big_cluster_centers.shape

(592L, 2L)


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)


In [73]:
offices = np.array ([[33.751277, -118.188740],
                     [25.867736, -80.324116],
                     [51.503016, -0.075479],
                     [52.378894, 4.885084],
                     [39.366487, 117.036146],
                     [-33.868457, 151.205134]
                    ])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [85]:
def distance(office, cluster_center):
    return np.linalg.norm(office - cluster_center), cluster_center

distance_list = []

for office in offices:
    for cluster_center in big_cluster_centers:
        distance_list.append(distance(office, cluster_center))
# сортируем, самый первый элемент списка будет 
list.sort(distance_list)        

In [101]:
#print distance_list.shape
print distance_list[0][1][0]

-33.86063042857143


In [103]:
with open('C3W1 Clustering MeanShift.txt', 'w') as f:
    f.write(str(distance_list[0][1][0]) + ' ' + str(distance_list[0][1][1]))